# MSP projekt 2 - Ondřej Lukášek (xlukas15)

## Úkol 1 - Věrohodnost

Začnu tím, že si naimportuji potřebné knihovny, se kterými budu v projektu pracovat.

In [7]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import scipy.stats as stats

Následně si načtu sheet, který obsahuje data, se kterými budu pracovat.

Předpřipravím si parametry tak, abych je mohl rovnou použít dál.

In [8]:
file_path = 'Data_2024.xlsx'
sheet_name = 'Data_věrohodnost'

data = pd.read_excel(file_path, sheet_name=sheet_name)
times = data['doba práce v oboru [roky]'].dropna().to_numpy()
censored = data['censored'].to_numpy()

initial_params = (1.5, 5.0)

### Bod 1

In [9]:
def weibull_log_likelihood(params, times, censored):
    k, lam = params
    
    # logaritmizace pro necenzurovana data
    log_f = np.log(k) - k * np.log(lam) + (k - 1) * np.log(times) - (times / lam)**k
    # logaritmizace pro cenzurovana data
    log_sf = - (times / lam)**k
    
    # spojeni logaritmu pro necenzurovana a cenzurovana data
    likelihood = (1 - censored) * log_f + censored * log_sf

    return likelihood.sum()


# MOZNA SMAZAT???
def weibull_log_likelihood_derivatives(params, times, censored):
    k, lam = params

    # predvypocitani konstant, co se budou v derivacich opakovat
    t_lam_k = (times / lam)**k
    log_t_lam = np.log(times / lam)
    
    # parcialni derivace logaritmu podle parametru k
    dL_dk = np.sum((1 - censored) * (1 / k + log_t_lam - t_lam_k * log_t_lam) + censored * (-t_lam_k * log_t_lam))
    # parcialni derivace logaritmu podle parametru lambda
    dL_dlam = np.sum((1 - censored) * (-k / lam + k * t_lam_k / lam) + censored * (k * t_lam_k / lam))
    
    return np.array([dL_dk, dL_dlam])


log_likelihood = weibull_log_likelihood(initial_params, times, censored)
derivatives = weibull_log_likelihood_derivatives(initial_params, times, censored)

print(f'Log-likelihood: {log_likelihood}')
print(f'Derivatives: {derivatives}')

Log-likelihood: -740.8976732574417
Derivatives: [64.24195393 77.00791218]


### Bod 2

In [ ]:
def neg_weibull_log_likelihood(params, times, censored):
    return -weibull_log_likelihood(params, times, censored)

result = opt.minimize(
    fun=neg_weibull_log_likelihood,
    x0=initial_params,
    args=(times, censored),
    method='L-BFGS-B',
    bounds=[(0.1, None), (0.1, None)]
)

optimal_shape, optimal_scale = result.x

print('Maximal likelihood estimation:')
print(f'Shape (k): {optimal_shape}')
print(f'Scale (lambda): {optimal_scale}')

Maximal likelihood estimation:
Shape (k): 6.172808847674017
Scale (lambda): 7.4294603426242265


### Bod 3

Pomocí věrohodnostního poměru otestujte hypotézu, že exponenciální rozdělení je postačujícím modelem zapsaných dat (Parametr tvaru = 1).

By default je distribuční funkce (pro $ x \geq 0 $) exponenciálního rozdělení:

$ f(x; \lambda) = \lambda e^{-\lambda x} $

Logaritmus potom bude:

$ \ell(x; \lambda) = \ln \lambda - \lambda x $

Log-likelihood potom je:

$ \ell(\lambda) = \sum_{i=1}^n \left[ \ln \lambda - \lambda x_i \right]  $

In [19]:
def exponential_log_likelihood(lam, times, censored):
    log_f = np.log(lam) - lam * times
    log_sf = -lam * times

    likelihood = (1 - censored) * log_f + censored * log_sf
    return np.sum(likelihood)


result_exp = opt.minimize(
    fun=lambda lam: -exponential_log_likelihood(lam[0], times, censored),
    x0=[1.0],
    bounds=[(0.1, None)],
    method='L-BFGS-B'
)

lambda_exp = result_exp.x[0]
log_likelihood_exp = -result_exp.fun

k_weibull, lambda_weibull = result.x
result_weibull = -result.fun

# VYPOCET VEROHODNOSTNIHO POMERU
LR = 2 * (result_weibull - log_likelihood_exp)

alpha = 0.05
critical_value = stats.chi2.ppf(1 - alpha, df=1)

print(f'Likelihood ratio: {LR}')
print(f'Critical value (alpha = {alpha}): {critical_value}')

if LR > critical_value:
    print('Zamítáme nulovou hypotézu. Exponenciální rozdělení není postačující.')
else:
    print('Nulovou hypotézu nezamítáme. Exponenciální rozdělení je postačující.')

Likelihood ratio: 592.3898153427439
Critical value (alpha = 0.05): 3.841458820694124
Zamítáme nulovou hypotézu. Exponenciální rozdělení není postačující.


### Bod 4

In [ ]:
# TODO

## Úkol 2 - Regrese

In [12]:
# TODO